In [1]:
# Useful additional packages 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import BasicAer, execute
from qiskit.quantum_info import Pauli, state_fidelity, basis_state, process_fidelity 

from qiskit.circuit import Parameter


from qiskit import *
IBMQ.load_accounts(hub=None)
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.ibmq import least_busy

In [4]:
#Simulation
#Misura di tutti i qubit
n=3
q = QuantumRegister(n, 'q')
c = ClassicalRegister(3, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i])
    


#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
print(circuits[-1].parameters)
circuits[-1].draw(output='mpl')

#circuits[1].draw()


job = execute(circ,
            backend=BasicAer.get_backend('qasm_simulator'),
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range])

phi_range3 = phi_range
counts3 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts3)
print('\n')
print('Counts of 000')
print(list(map(lambda c: 1/1024*c.get('000', 0), counts3)))

Phi range
[0.0, 0.7853981633974483, 1.5707963267948966, 2.356194490192345, 3.141592653589793, 3.9269908169872414, 4.71238898038469, 5.497787143782138]
set()


Counts
[{'000': 1024}, {'000': 130, '001': 894}, {'000': 521, '001': 503}, {'000': 861, '001': 163}, {'001': 1024}, {'000': 870, '001': 154}, {'000': 536, '001': 488}, {'000': 163, '001': 861}]


Counts of 000
[1.0, 0.126953125, 0.5087890625, 0.8408203125, 0.0, 0.849609375, 0.5234375, 0.1591796875]


In [6]:
#Run in real quantum computer
#Misura di tutti i qubit
n = 3
q = QuantumRegister(n, 'q')
c = ClassicalRegister(3, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')


job = execute(circ,
            backend=IBMQ.get_backend('ibmqx4'),
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range])

job_monitor(job)

phi_range3 = phi_range
counts3 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts3)
print('\n')
print('Counts of 000')
print(list(map(lambda c: 1/1024*c.get('000', 0), counts3)))

Phi range
[0.0, 0.7853981633974483, 1.5707963267948966, 2.356194490192345, 3.141592653589793, 3.9269908169872414, 4.71238898038469, 5.497787143782138]
Job Status: job is queued (20)      

JobError: "HTTPSConnectionPool(host='quantumexperience.ng.bluemix.net', port=443): Max retries exceeded with url: /api/Jobs/5d3bf6ec34937d0075bb0427/status?access_token=IdELXkvrO4jpBch5DJFvpImy08XQcRz3mzCfIGjBUOeFBecpSY6FoS9JqEjLGNiM (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x123b5e898>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))"